In [2]:
import pandas as pd
import numpy as np
import requests
import json

In [26]:
def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file._append(temp_df)
    file.to_csv(filename + '.csv')

In [6]:
!pip install geopy


In [8]:
import geopy


In [10]:
from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [12]:
def dist_from_location(house, location):
    """
    this function finds the distance of a location from the 1st address
    First is a dataframe with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    Second is tuple with latitude and longitude of location
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['',100]
        distance = geodesic(flat_loc,location)
        distance = float(str(distance)[:-3]) # convert to float
        flat_amenity[0] = flat
        flat_amenity[1] = distance
        results[flat] = flat_amenity
    return results

In [16]:
# price1999 = pd.read_csv('Project 3 - Real Estate - Business Analytics/OutsideDataSource/Resale_data_dataGovSg/ResaleFlatPricesBasedonApprovalDate19901999.csv')
# price2012 = pd.read_csv('Project 3 - Real Estate - Business Analytics/OutsideDataSource/Resale_data_dataGovSg/ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv')
price2014 = pd.read_csv('Project 3 - Real Estate - Business Analytics/OutsideDataSource/Resale_data_dataGovSg/ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv')
# price2016 = pd.read_csv('Project 3 - Real Estate - Business Analytics/OutsideDataSource/Resale_data_dataGovSg/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv')
price2017 = pd.read_csv('Project 3 - Real Estate - Business Analytics/OutsideDataSource/Resale_data_dataGovSg/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')

prices = pd.concat([price2014], sort=False)
prices = pd.concat([prices, price2017], axis=0, ignore_index=True, sort=False)

In [18]:
prices = prices[['block', 'street_name']]
prices['address'] = prices['block'] + ' ' + prices['street_name']
all_address = list(prices['address'])
unique_address = list(set(all_address))

print('Unique addresses:', len(unique_address))

Unique addresses: 9629


In [20]:
unique_address[:10]

['17 TIONG BAHRU RD',
 '565 CHOA CHU KANG ST 52',
 '181 YUNG SHENG RD',
 "89 C'WEALTH DR",
 '215A COMPASSVALE DR',
 '494 JURONG WEST ST 41',
 '216 MARSILING LANE',
 '872 YISHUN ST 81',
 "95 C'WEALTH DR",
 '7 JLN BT MERAH']

In [28]:
find_postal(unique_address, 'Project 3 - Real Estate - Business Analytics/OutsideDataSource/Resale_data_dataGovSg/flat_coordinates2')

0 https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=17 TIONG BAHRU RD
1 https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=565 CHOA CHU KANG ST 52
2 https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=181 YUNG SHENG RD
3 https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=89 C'WEALTH DR
4 https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=215A COMPASSVALE DR
5 https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=494 JURONG WEST ST 41
6 https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=216 MARSILING LANE
7 https://www.onemap.gov.sg/api/common/elastic/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=872 YISHUN ST 81
8 https://www.o

ReadTimeout: HTTPSConnectionPool(host='www.onemap.gov.sg', port=443): Read timed out. (read timeout=None)

In [ ]:
# find_postal(unique_address, 'Data/flat_coordinates')


In [ ]:
flat_coord = pd.read_csv('Data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head()

In [ ]:
supermarket = pd.read_csv('Data/list-of-supermarket-licences.csv')
supermarket.head()

In [ ]:
supermerket_address = list(supermarket['postal_code'])
unique_supermarket_address = list(set(supermerket_address))

print('Unique addresses:', len(unique_supermarket_address))